In [1]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
import os
os.chdir('/content/drive/My Drive/ThesisProject')

%ls

In [ ]:
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

data = pd.read_csv('german-ai-companies-crunchbase.csv')
data

In [ ]:
# Removes any occurrence of the string "Logo " and any characters before it in the 'OrganizationName' column using regex.
data['OrganizationName'] = data['OrganizationName'].str.replace(r'.*Logo\s*', '', regex=True)

# Drops duplicate rows based on the 'OrganizationName' column, keeping the first occurrence.
data = data.drop_duplicates(subset='OrganizationName', keep='first')

# Sorts the DataFrame by the 'OrganizationName' column in ascending order.
data = data.sort_values(by='OrganizationName', ascending=True)

# Removes rows where 'OrganizationName' is NaN.
data = data.dropna(subset=['OrganizationName'])

# Splits the 'HeadquartersLocation' column into 'HeadquartersCity', 'HeadquartersState', and a temporary column '_'
# based on the delimiter ' , ', and selects the first three splits for the new columns.
data[['HeadquartersCity', 'HeadquartersState', '_']] = data['HeadquartersLocation'].str.split(' , ', expand=True).iloc[:, :3]

# Drops the 'HeadquartersLocation' column as it has been split into new columns.
data = data.drop(columns=['HeadquartersLocation'])

# Drops the temporary column '_' used in the splitting process.
data = data.drop(columns=['_'])

# Drops the 'CBRank' column from the DataFrame.
data = data.drop(columns=['CBRank'])

# Drops the 'Unnamed: 25' column from the DataFrame.
data = data.drop(columns=['Unnamed: 25'])

# Concatenates the 'FullDescription' and 'Description' columns, separated by a space, and stores the result in 'FullDescription'.
data['FullDescription'] = data['FullDescription'] + " " + data['Description']

# Drops the 'Description' column as its content has been merged into 'FullDescription'.
data = data.drop(columns=['Description'])

# Removes the string 'Artificial Intelligence (AI)' from the 'Industries' column, then strips leading and trailing whitespace.
data['Industries'] = data['Industries'].str.replace('Artificial Intelligence (AI)', '', regex=False).str.strip()

# Renames the 'Industries' column to 'OtherIndustries'.
data = data.rename(columns={'Industries': 'OtherIndustries'})

# Replaces sequences of commas surrounded by spaces with a single comma and space.
data['OtherIndustries'] = data['OtherIndustries'].str.replace(r'\s*,\s*,\s*', ', ', regex=True)

# Strips leading and trailing commas and spaces from the 'OtherIndustries' column.
data['OtherIndustries'] = data['OtherIndustries'].str.strip(', ')

# Removes any spaces before commas in the 'OtherIndustries' column.
data['OtherIndustries'] = data['OtherIndustries'].str.replace(r'^,\s*|\s*,$', '', regex=True)

# Further ensures no spaces exist before commas in 'OtherIndustries'.
data['OtherIndustries'] = data['OtherIndustries'].str.replace(r'\s+(,)', r'\1', regex=True)

# Extracts the year from 'FoundedDate' and converts it to an integer.
data['FoundedDate'] = data['FoundedDate'].str.extract(r'(\d{4})')[0].astype('Int64')

# Similar to 'FoundedDate', extracts and converts the year in 'ExitDate' to an integer.
data['ExitDate'] = data['ExitDate'].str.extract(r'(\d{4})')[0].astype('Int64')

# Extracts and converts the year in 'ClosedDate' to an integer.
data['ClosedDate'] = data['ClosedDate'].str.extract(r'(\d{4})')[0].astype('Int64')

# Extracts and converts the year in 'LastFundingDate' to an integer.
data['LastFundingDate'] = data['LastFundingDate'].str.extract(r'(\d{4})')[0].astype('Int64')

# Renames the 'Company Type' column to 'CompanyType'.
data = data.rename(columns={'Company Type': 'CompanyType'})

# Renames 'FoundedDate' to 'FoundedYear' for clarity.
data = data.rename(columns={'FoundedDate': 'FoundedYear'})

# Renames 'ExitDate' to 'ExitYear' for clarity.
data = data.rename(columns={'ExitDate': 'ExitYear'})

# Renames 'ClosedDate' to 'ClosedYear' for clarity.
data = data.rename(columns={'ClosedDate': 'ClosedYear'})

# Renames 'LastFundingDate' to 'LastFundingYear' for clarity.
data = data.rename(columns={'LastFundingDate': 'LastFundingYear'})

# Ensures 'PostalCode' is treated as a string, necessary for the subsequent extraction.
data['PostalCode'] = data['PostalCode'].astype(str)

# Extracts up to the last five digits from 'PostalCode' and discards any non-digit characters before them.
data['PostalCode'] = data['PostalCode'].str.extract('(\d{1,5})\D*$')[0]

# Converts 'PostalCode' to a nullable integer type, preserving NaN values for missing or invalid data.
data['PostalCode'] = data['PostalCode'].astype('Int64')

# Converts 'NumberOfFounders' to a nullable integer type, coercing errors to NaN.
data['NumberOfFounders'] = pd.to_numeric(data['NumberOfFounders'], errors='coerce').astype('Int64')

# Converts 'NumberOfEmployees' to a nullable integer type, coercing errors to NaN.
data['NumberOfEmployees'] = pd.to_numeric(data['NumberOfEmployees'], errors='coerce').astype('Int64')

# Converts 'NumberOfInvestors' to a nullable integer type, coercing errors to NaN.
data['NumberOfInvestors'] = pd.to_numeric(data['NumberOfInvestors'], errors='coerce').astype('Int64')

# Converts 'NumberOfFundingRounds' to a nullable integer type, coercing errors to NaN.
data['NumberOfFundingRounds'] = pd.to_numeric(data['NumberOfFundingRounds'], errors='coerce').astype('Int64')

# Saves Cleaned dataset to
data.to_csv('german-ai-companies-crunchbase-cleaned.csv', index=False)

In [ ]:
print(data.columns)
print(data.describe(include='all'))
data.dtypes

In [ ]:
# Define columns with 0, 1, or 2 rows of missing data
columns_with_few_missing = [
    'EstimatedRevenueRange', 'OperatingStatus', 'CompanyType', 'Website',
    'TotalEquityFundingAmount', 'TotalFundingAmount', 'Top5Investors',
    'FullDescription', 'LastFundingAmount', 'FundingStatus', 'LastFundingType', 'HeadquartersState'
]

# Drop rows where any of the specified columns have missing data
new_data = data.dropna(subset=columns_with_few_missing)
new_data['EstimatedRevenueRange'] = new_data['EstimatedRevenueRange'].replace('—', np.nan)
new_data['TotalFundingAmount'] = new_data['TotalFundingAmount'].replace('—', np.nan)
new_data['LastFundingAmount'] = new_data['LastFundingAmount'].replace('—', np.nan)

#columns_with_modifier = [
#    'PostalCode', 'FoundedYear', 'NumberOfFounders', 'NumberOfFundingRounds',
#    'LastFundingYear', 'EstimatedRevenueRange', 'TotalFundingAmount', 'LastFundingAmount'
#]
#new_data = new_data.dropna(subset=columns_with_modifier)

new_data = new_data.drop(columns=['NumberOfEmployees'])
new_data = new_data.drop(columns=['NumberOfInvestors'])
new_data = new_data.drop(columns=['TotalEquityFundingAmount'])
new_data = new_data.drop(columns=['ExitYear'])
new_data = new_data.drop(columns=['ClosedYear'])
new_data = new_data.drop(columns=['CompanyType'])

missing_data_count_new = new_data.isnull().sum()
missing_data_count_new

In [ ]:
new_data['EstimatedRevenueRange'] = new_data['EstimatedRevenueRange'].astype('category')
def add_www(url):
    if not url.startswith('www.'):
        return 'www.' + url
    return url

# Apply the function to the 'Website' column
new_data['Website'] = new_data['Website'].apply(add_www)

In [ ]:
conversion_rates = {
    '€': 1.086,  # EUR to USD
    '£': 1.260,  # GBP to USD
    'CN¥': 0.140,  # CNY to USD
    '¥': 0.140,   # Handle potential CNY representation
    'SEK': 0.093, # SEK to USD (example rate, adjust as needed)
    'kr': 0.093   # SEK symbol variant to USD (ensure correct conversion rate)
}

# Function to convert currencies to USD
def convert_to_usd(amount):
    # Check if amount is NaN and return NaN if it is
    if pd.isna(amount):
        return np.nan

    # Convert amount to string if it's not already
    if not isinstance(amount, str):
        amount = str(amount)

    # Use regular expressions to extract symbol and amount
    match = re.match(r'^\s*([A-Za-z¥€£$]*?)\s*([\d,.]+)\s*([A-Za-z¥€£$]*?)\s*$', amount)

    if not match:
        raise ValueError(f"Unsupported format: {amount}")

    symbol_before = match.group(1).strip()
    value = match.group(2).replace(',', '')  # Remove commas for thousands
    symbol_after = match.group(3).strip()

    # Determine the currency symbol
    symbol = symbol_before if symbol_before else symbol_after
    if not symbol:
        raise ValueError(f"Unsupported currency: {amount}")

    # Convert string to float
    value = float(value)

    # Convert to USD using static conversion rates
    if symbol in conversion_rates:
        return value * conversion_rates[symbol]
    elif symbol == '$':
        return value  # Already in USD
    else:
        raise ValueError(f"Unsupported currency: {symbol}")

# Example usage with the new_data DataFrame
new_data['LastFundingAmount_USD'] = new_data['LastFundingAmount'].apply(convert_to_usd)
new_data['TotalFundingAmount_USD'] = new_data['TotalFundingAmount'].apply(convert_to_usd)

# Drop the original columns if needed
new_data = new_data.drop(columns=['LastFundingAmount', 'TotalFundingAmount'])

# Checking how many companies have a founding year >= 2008
new_data = new_data[new_data['FoundedYear'] >= 2008]

In [ ]:
file_path = 'german-ai-startups.xlsx'

# Replace problematic characters with an empty string
for col in new_data.columns:
    if new_data[col].dtype == 'object':
        new_data[col] = new_data[col].str.replace('\x03', '', regex=True)

new_data.to_excel(file_path, index=False)

new_data = pd.read_excel('german-ai-startups-2.xlsx')

In [ ]:
df = new_data.copy()
# Drop the column
df.info()

In [ ]:
# Display descriptive statistics for all columns
descriptive_stats = df.describe(include='all')

# Convert descriptive statistics to a dataframe for better visualization
descriptive_stats_df = pd.DataFrame(descriptive_stats)
print(descriptive_stats_df)

# Plotting the distribution of 'EstimatedRevenueRange'
plt.figure(figsize=(10, 6))
sns.countplot(data=df, y='EstimatedRevenueRange', palette='viridis')
plt.title('Distribution of Estimated Revenue Range')
plt.xlabel('Count')
plt.ylabel('Estimated Revenue Range')
plt.grid(axis='x')

# Calculate the total number of instances for percentage calculation
total = len(df['EstimatedRevenueRange'])

# Adding count and percentage labels
for p in plt.gca().patches:
    count = int(p.get_width())
    percentage = f'{count / total * 100:.1f}%'
    plt.text(p.get_width() + 0.3, p.get_y() + p.get_height() / 2,
             f'{count}\n({percentage})', ha='center', va='center', fontsize=10, color='black', weight='bold')
plt.show()

# Function to display both number and percentage in pie chart
def func(pct, allvalues):
    absolute = int(round(pct/100.*sum(allvalues)))
    return "{:d}\n({:.1f}%)".format(absolute, pct)

# Plotting the distribution of 'OperatingStatus'
plt.figure(figsize=(8, 4))
df['OperatingStatus'].value_counts().plot(kind='pie', autopct=lambda pct: func(pct, df['OperatingStatus'].value_counts()),
                                          colors=sns.color_palette('pastel'), textprops={'fontsize': 12})
plt.title('Operating Status Distribution')
plt.ylabel('')
plt.show()

# Plotting the 'EndUser' distribution as a pie chart (assuming the column name)
if 'EndUser' in df.columns:
    plt.figure(figsize=(8, 8))
    df['EndUser'].value_counts().plot(kind='pie', autopct=lambda pct: func(pct, df['EndUser'].value_counts()),
                                      colors=sns.color_palette('coolwarm'), textprops={'fontsize': 12})
    plt.title('End User Distribution')
    plt.ylabel('')
    plt.show()
else:
    print("The column 'EndUser' does not exist in the dataset.")

# Plotting 'FoundedYear' distribution
plt.figure(figsize=(12, 6))
sns.histplot(data=df, x='FoundedYear', bins=20, kde=True, color='blue')
plt.title('Distribution of Founded Year')
plt.xlabel('Founded Year')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

# Plotting Top 10 States by Number of Startups
top_10_headquarters_states = df['HeadquartersState'].value_counts().head(10)

# Creating the horizontal bar plot
plt.figure(figsize=(12, 8))
sns.barplot(x=top_10_headquarters_states.values, y=top_10_headquarters_states.index, color='skyblue')
plt.title('Top 10 States by Number of Startups')
plt.xlabel('Number of Startups')
plt.ylabel('State of Headquarters')
plt.grid(axis='x')

# Adding count labels to the bars
for i, v in enumerate(top_10_headquarters_states):
    plt.text(v + 0.3, i, str(v), color='black', va='center', fontsize=10)

plt.show()

In [ ]:
# Split the 'OtherIndustries' column into individual industries and count occurrences
industry_series = df['OtherIndustries'].str.split(',').explode().str.strip()
industry_counts = industry_series.value_counts()

# Calculate average funding by industry
industry_funding_df = df.dropna(subset=['TotalFundingAmount_USD'])  # Drop rows with missing funding amounts
industry_funding_series = industry_funding_df['OtherIndustries'].str.split(',').explode().str.strip()
industry_funding_combined = pd.DataFrame({
    'Industry': industry_funding_series,
    'Funding': industry_funding_df.loc[industry_funding_series.index, 'TotalFundingAmount_USD'].values
})
average_funding_by_industry = industry_funding_combined.groupby('Industry')['Funding'].mean().sort_values(ascending=False)

# Top 10 and Bottom 10 Other Industries by Number of Startups
top_10_industries_count = industry_counts.head(10)
bottom_10_industries_count = industry_counts.tail(10)

# Plotting Top 10 Other Industries by Number of Startups
plt.figure(figsize=(14, 8))
sns.barplot(x=top_10_industries_count.values, y=top_10_industries_count.index, palette='viridis')
plt.title('Top 10 Other Industries by Number of Startups')
plt.xlabel('Number of Startups')
plt.ylabel('Industry')
plt.grid(axis='x')
for i, v in enumerate(top_10_industries_count):
    plt.text(v + 0.5, i, str(v), color='black', va='center', fontsize=10)
plt.show()

# Calculate median funding by industry with at least 3 companies
industry_funding_combined['Count'] = industry_funding_combined.groupby('Industry')['Industry'].transform('count')
industry_funding_filtered = industry_funding_combined[industry_funding_combined['Count'] >= 3]

# Calculate the median funding amount by industry
median_funding_by_industry = industry_funding_filtered.groupby('Industry')['Funding'].median().sort_values(ascending=False)

# Top 10 and Bottom 10 Industries by Median Funding
top_10_industries_median_funding = median_funding_by_industry.head(10)
bottom_10_industries_median_funding = median_funding_by_industry.tail(10)

# Plotting Top 10 Industries by Median Funding
plt.figure(figsize=(14, 8))
sns.barplot(x=top_10_industries_median_funding.values, y=top_10_industries_median_funding.index, palette='Blues_d')
plt.title('Top 10 Other Industries by Median Funding (min. 3 companies)')
plt.xlabel('Median Funding Amount (USD)')
plt.ylabel('Industry')
plt.grid(axis='x')
for i, v in enumerate(top_10_industries_median_funding):
    count = int(industry_funding_filtered[industry_funding_filtered['Industry'] == top_10_industries_median_funding.index[i]]['Count'].iloc[0])
    plt.text(v + 0.05 * v, i, f'${v:,.0f}\n({count} companies)', color='black', va='center', fontsize=10)
plt.show()

# Plotting Bottom 10 Industries by Median Funding
plt.figure(figsize=(14, 8))
sns.barplot(x=bottom_10_industries_median_funding.values, y=bottom_10_industries_median_funding.index, palette='Reds_d')
plt.title('Bottom 10 Other Industries by Median Funding (min. 3 companies)')
plt.xlabel('Median Funding Amount (USD)')
plt.ylabel('Industry')
plt.grid(axis='x')
for i, v in enumerate(bottom_10_industries_median_funding):
    count = int(industry_funding_filtered[industry_funding_filtered['Industry'] == bottom_10_industries_median_funding.index[i]]['Count'].iloc[0])
    plt.text(v + 0.05 * v, i, f'${v:,.0f}\n({count} companies)', color='black', va='center', fontsize=10)
plt.show()

In [ ]:
# Analyzing sectoral data based on the 'IndustryGroup' column

# Top 10 Industry Groups
top_10_industry_groups = df['IndustryGroup'].value_counts().head(10)

# Plotting Top 10 Industry Groups
plt.figure(figsize=(14, 8))
sns.barplot(x=top_10_industry_groups.values, y=top_10_industry_groups.index, palette='magma')
plt.title('Top 10 Industry Groups by Number of Startups')
plt.xlabel('Number of Startups')
plt.ylabel('Industry Group')
for i, v in enumerate(top_10_industry_groups):
    plt.text(v + 0.5, i, str(v), color='black', va='center')
plt.grid(axis='x')
plt.show()

# Calculating median funding by industry group with at least 3 companies
df['IndustryGroupCount'] = df.groupby('IndustryGroup')['IndustryGroup'].transform('count')
industry_group_filtered = df[df['IndustryGroupCount'] >= 3]

# Median Funding Amount by Industry Group
median_funding_by_industry_group = industry_group_filtered.groupby('IndustryGroup')['TotalFundingAmount_USD'].median().sort_values(ascending=False).head(10)

# Plotting Median Funding Amount by Industry Group
plt.figure(figsize=(14, 8))
sns.barplot(x=median_funding_by_industry_group.values, y=median_funding_by_industry_group.index, palette='Blues_d')
plt.title('Median Funding Amount by Industry Group (Top 10, min. 3 startups)')
plt.xlabel('Median Funding Amount (USD)')
plt.ylabel('Industry Group')
plt.grid(axis='x')
for i, v in enumerate(median_funding_by_industry_group):
    count = industry_group_filtered[industry_group_filtered['IndustryGroup'] == median_funding_by_industry_group.index[i]]['IndustryGroupCount'].iloc[0]
    plt.text(v + 0.5, i, f'${v:,.0f}\n({count} startups)', color='black', va='center', fontsize=10)
plt.show()

# Calculating average number of funding rounds by industry group with at least 3 companies
avg_funding_rounds_by_industry_group = industry_group_filtered.groupby('IndustryGroup')['NumberOfFundingRounds'].mean().sort_values(ascending=False).head(10)

# Plotting Average Number of Funding Rounds by Industry Group
plt.figure(figsize=(14, 8))
sns.barplot(x=avg_funding_rounds_by_industry_group.values, y=avg_funding_rounds_by_industry_group.index, palette='cubehelix')
plt.title('Average Number of Funding Rounds by Industry Group (Top 10, min. 3 startups)')
plt.xlabel('Average Number of Funding Rounds')
plt.ylabel('Industry Group')
plt.grid(axis='x')
for i, v in enumerate(avg_funding_rounds_by_industry_group):
    count = industry_group_filtered[industry_group_filtered['IndustryGroup'] == avg_funding_rounds_by_industry_group.index[i]]['IndustryGroupCount'].iloc[0]
    plt.text(v + 0.02, i, f'{v:.2f}\n({count} startups)', color='black', va='center', fontsize=10)
plt.show()

